# Debug Drill: The Unscaled Lasso

**Scenario:**
A colleague is using Lasso regression for feature selection.

"Lasso says tenure is unimportant!" they report. "We should remove it."

But you know tenure is a strong predictor. What's going on?

**Your Task:**
1. Run the Lasso and see the unexpected feature selection
2. Identify why Lasso is giving wrong results
3. Fix the pipeline and compare
4. Write a 3-bullet postmortem

---

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import Lasso, LassoCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error

np.random.seed(42)

In [ ]:
# Generate data with features on VERY different scales
n = 500

# Features on wildly different scales
tenure_days = np.random.uniform(30, 1000, n)    # Range: ~30-1000
monthly_spend = np.random.uniform(20, 200, n)   # Range: ~20-200
orders = np.random.poisson(5, n)                # Range: ~0-15
is_premium = np.random.binomial(1, 0.3, n)      # Range: 0-1

# True relationship (tenure is IMPORTANT)
true_coefs = {
    'tenure_days': 0.5,      # Important!
    'monthly_spend': 2.0,    # Important!
    'orders': 30.0,          # Important!
    'is_premium': 100.0      # Important!
}

ltv = (
    true_coefs['tenure_days'] * tenure_days +
    true_coefs['monthly_spend'] * monthly_spend +
    true_coefs['orders'] * orders +
    true_coefs['is_premium'] * is_premium +
    np.random.normal(0, 50, n)  # Noise
)

df = pd.DataFrame({
    'tenure_days': tenure_days,
    'monthly_spend': monthly_spend,
    'orders': orders,
    'is_premium': is_premium,
    'ltv': ltv
})

print("=== Feature Scales ===")
print(df.describe().loc[['mean', 'std', 'min', 'max']].T.round(1))

In [ ]:
# ===== COLLEAGUE'S CODE (BUG: NO SCALING) =====

feature_cols = ['tenure_days', 'monthly_spend', 'orders', 'is_premium']
X = df[feature_cols]
y = df['ltv']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Lasso without scaling (BUG!)
lasso_unscaled = Lasso(alpha=1.0)
lasso_unscaled.fit(X_train, y_train)  # <-- No scaling!

print("=== Colleague's Lasso Results (Unscaled) ===")
print("\nCoefficients:")
for name, coef in zip(feature_cols, lasso_unscaled.coef_):
    status = "ZEROED" if abs(coef) < 0.01 else f"{coef:.3f}"
    true_val = true_coefs[name]
    print(f"  {name:<15}: {status:<10} (true: {true_val})")

print(f"\n❌ Colleague's conclusion: 'tenure_days is unimportant, remove it!'")

---

## Your Investigation

### Step 1: Understand why this happened

In [ ]:
print("=== Why Lasso Gets This Wrong ===")
print()
print("Lasso penalizes the SUM of |coefficients|.")
print("When features are on different scales, this penalty is UNFAIR.")
print()
print("Consider:")
for name in feature_cols:
    scale = df[name].std()
    true_coef = true_coefs[name]
    contribution = scale * true_coef
    print(f"  {name:<15}: std={scale:>6.1f}, true_coef={true_coef:>5.1f}, contribution≈{contribution:>6.0f}")

print()
print("💡 tenure_days has a SMALL coefficient (0.5) but high variance.")
print("   Lasso sees a small coefficient and zeros it out!")
print("   But the actual CONTRIBUTION to prediction is large.")

In [ ]:
# Visualize the scale problem
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Unscaled coefficients
ax1 = axes[0]
ax1.bar(feature_cols, lasso_unscaled.coef_, color=['red' if abs(c) < 0.01 else 'blue' for c in lasso_unscaled.coef_])
ax1.axhline(y=0, color='gray', linestyle='--')
ax1.set_ylabel('Coefficient')
ax1.set_title('Lasso Coefficients (UNSCALED) - tenure wrongly zeroed!')
ax1.tick_params(axis='x', rotation=45)

# True coefficients
ax2 = axes[1]
ax2.bar(feature_cols, [true_coefs[f] for f in feature_cols], color='green', alpha=0.7)
ax2.axhline(y=0, color='gray', linestyle='--')
ax2.set_ylabel('Coefficient')
ax2.set_title('TRUE Coefficients - tenure IS important!')
ax2.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

### Step 2: TODO - Fix with scaling

In [ ]:
# TODO: Scale features BEFORE Lasso

# Uncomment and complete:

# # Step 1: Scale the features (fit on train only!)
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)
# 
# # Step 2: Fit Lasso on scaled data
# lasso_scaled = LassoCV(cv=5)  # Use CV to find best alpha
# lasso_scaled.fit(X_train_scaled, y_train)
# 
# print("=== Fixed Lasso Results (Scaled) ===")
# print(f"\nOptimal alpha: {lasso_scaled.alpha_:.4f}")
# print("\nCoefficients:")
# for name, coef in zip(feature_cols, lasso_scaled.coef_):
#     status = "ZEROED" if abs(coef) < 0.01 else f"{coef:.3f}"
#     print(f"  {name:<15}: {status}")

In [ ]:
# TODO: Compare performance

# Uncomment:

# r2_unscaled = r2_score(y_test, lasso_unscaled.predict(X_test))
# r2_scaled = r2_score(y_test, lasso_scaled.predict(X_test_scaled))
# 
# print("=== Performance Comparison ===")
# print(f"\nUnscaled Lasso:")
# print(f"  R²: {r2_unscaled:.3f}")
# print(f"  Features selected: {(np.abs(lasso_unscaled.coef_) > 0.01).sum()}")
# print(f"\nScaled Lasso:")
# print(f"  R²: {r2_scaled:.3f}")
# print(f"  Features selected: {(np.abs(lasso_scaled.coef_) > 0.01).sum()}")

In [ ]:
# ============================================
# SELF-CHECK
# ============================================

# Uncomment:

# # After scaling, tenure should NOT be zeroed
# tenure_idx = feature_cols.index('tenure_days')
# assert abs(lasso_scaled.coef_[tenure_idx]) > 0.01, "tenure_days should not be zeroed after scaling!"
# assert r2_scaled > r2_unscaled, "Scaled Lasso should have better R²"
# 
# print("✓ Lasso fixed!")
# print(f"✓ tenure_days now has coefficient: {lasso_scaled.coef_[tenure_idx]:.3f}")
# print(f"✓ R² improved: {r2_unscaled:.3f} → {r2_scaled:.3f}")

### Step 3: Write your postmortem

In [ ]:
postmortem = """
## Postmortem: The Unscaled Lasso

### What happened:
- (Your answer: What feature was wrongly eliminated?)

### Root cause:
- (Your answer: Why does Lasso need scaled features?)

### How to prevent:
- (Your answer: What preprocessing is required for Lasso/Ridge?)

"""

print(postmortem)

---

## ✅ Drill Complete!

**Key lessons:**

1. **Always scale before regularization.** Lasso/Ridge penalize coefficient magnitude, so features must be comparable.

2. **Unscaled Lasso makes unfair comparisons.** Features with large scales get artificially small coefficients.

3. **Feature selection without scaling is wrong.** You'll drop important features just because they're on a different scale.

4. **Standard scaling (mean=0, std=1) is the norm.** Makes all features comparable.

---

## When Scaling Matters

| Model | Scaling Required? | Why |
|-------|-------------------|-----|
| Linear Regression | Optional | Doesn't affect predictions, only coefficient interpretation |
| Lasso/Ridge | **Required** | Penalty applied to coefficient magnitude |
| Logistic Regression | Recommended | Helps convergence, required for regularization |
| SVM | **Required** | Distance-based algorithm |
| k-NN | **Required** | Distance-based algorithm |
| Decision Trees | Not needed | Splits are scale-invariant |
| Random Forest | Not needed | Ensemble of trees |